# Non supervised text topic detection

## LDA with Gensim

In [1]:
import os
import re
import string

#import pandas as pd

data = '../data'

In [2]:
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train',
                 remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
print(len(twenty_train['data']))

11314


In [3]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer

def tokenize_doc(text):
    sents = sent_tokenize(text)
    tokens = []
    for s in sents:
        for t in TreebankWordTokenizer().tokenize(s):
            if len(t)<2 or '\n' in t or '\t' in t or t.isnumeric():
                continue
            else:
                tokens += [t.lower()]
    return tokens

In [4]:
%%time

docs_tokenized = []
for text in twenty_train['data']:
    docs_tokenized += [tokenize_doc(text)]
print(len(docs_tokenized))

11314
CPU times: user 24 s, sys: 265 ms, total: 24.3 s
Wall time: 24.5 s


In [6]:
# Get the NLTK stopwords list
import nltk

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

In [11]:
# create dictionary and bag of words corpus

from gensim.corpora import Dictionary, MmCorpus
    
print('Building dictionary...')
dictionary = Dictionary(docs_tokenized)

# Stopwords

additional_stopwords=set(['...', '\'s','\'\'',  "``", "n\'t", "\'re", "\'m", "\'ve",
                          '--', '"', "\'ax", 'max', 'q,3'])

#additional_stopwords=set()

stopwords = nltk_stopwords().union(additional_stopwords)
stopword_ids = map(dictionary.token2id.get, stopwords)
dictionary.filter_tokens(stopword_ids)
dictionary.compactify()

# Filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.95, keep_n=None)
dictionary.compactify()

print('Building corpus...')
corpus = [dictionary.doc2bow(doc) for doc in docs_tokenized]

Building dictionary...
Building corpus...


In [12]:
%%time

# Train the LDA model

from gensim import models

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=25, passes=10)
                                      
lda.save(os.path.join(data,'newsgroups_50_lda.model'))

CPU times: user 1min 6s, sys: 1.73 s, total: 1min 7s
Wall time: 1min 7s


In [13]:
lda.show_topics()

[(4,
  '0.021*"mr." + 0.019*"president" + 0.013*"stephanopoulos" + 0.007*"tax" + 0.007*"going" + 0.006*"jobs" + 0.005*"year" + 0.005*"states" + 0.005*"american" + 0.005*"new"'),
 (2,
  '0.020*"god" + 0.016*"people" + 0.010*"would" + 0.009*"one" + 0.008*"jesus" + 0.007*"us" + 0.006*"say" + 0.005*"life" + 0.005*"even" + 0.005*"believe"'),
 (5,
  '0.022*"year" + 0.017*"gm" + 0.015*"player" + 0.014*"season" + 0.012*"players" + 0.012*"good" + 0.011*"last" + 0.010*"team" + 0.010*"john" + 0.008*"mike"'),
 (8,
  '0.016*"government" + 0.013*"gun" + 0.012*"law" + 0.011*"state" + 0.009*"right" + 0.009*"rights" + 0.008*"public" + 0.007*"guns" + 0.007*"control" + 0.007*"would"'),
 (14,
  '0.010*"software" + 0.010*"data" + 0.010*"use" + 0.009*"image" + 0.007*"computer" + 0.007*"color" + 0.007*"internet" + 0.006*"system" + 0.006*"images" + 0.006*"available"'),
 (24,
  '0.015*"israel" + 0.014*"war" + 0.012*"turkish" + 0.012*"jews" + 0.012*"armenian" + 0.010*"israeli" + 0.009*"armenians" + 0.008*"peopl

In [20]:
lda.get_document_topics(corpus[:10])

In [19]:
for i in range(lda.num_topics):
    print(i, lda.show_topic(i))
    print('\n')

0 [('one', 0.014967264), ('evidence', 0.007125512), ('true', 0.006811183), ('believe', 0.006538611), ('would', 0.0064531323), ('point', 0.005327043), ('book', 0.0052255443), ('many', 0.005205821), ('exist', 0.0048122657), ('claim', 0.0047982084)]


1 [('article', 0.010416013), ('master', 0.009780715), ('read', 0.0069780317), ('wiring', 0.006480035), ('posting', 0.0058471775), ('may', 0.005492524), ('context', 0.0054858625), ('given', 0.0050711967), ('faq', 0.0049202973), ('part', 0.0048426054)]


2 [('god', 0.020054193), ('people', 0.015518317), ('would', 0.00987888), ('one', 0.008836058), ('jesus', 0.008279402), ('us', 0.0074752066), ('say', 0.0056957416), ('life', 0.005287182), ('even', 0.0048442464), ('believe', 0.004740599)]


3 [('thanks', 0.020954615), ('please', 0.02049899), ('anyone', 0.016487125), ('would', 0.013559956), ('know', 0.012694959), ('email', 0.009846773), ('help', 0.0094499495), ('send', 0.00923283), ('e-mail', 0.00881854), ('information', 0.0074023246)]


4 [('mr.

# Visualization

In [14]:
import pyLDAvis

pyLDAvis.enable_notebook()

In [15]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, corpus, dictionary)

/Users/jorge/anaconda3/envs/tm/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
13     16.201761        1       1  0.178977 -0.145937
2      10.967565        1       2  0.147113 -0.147092
12      6.534413        1       3  0.146527  0.163883
0       5.884079        1       4  0.171094 -0.041182
3       5.222198        1       5  0.123879  0.093548
23      5.006221        1       6  0.042499  0.079160
22      4.899346        1       7  0.060432 -0.214409
10      4.625042        1       8  0.123587  0.067300
8       4.025807        1       9  0.059929  0.030350
6       4.025583        1      10  0.145991  0.068244
24      3.579009        1      11 -0.003011 -0.092569
14      3.443527        1      12  0.099441  0.209209
4       3.435462        1      13  0.036170 -0.091772
17      3.164975        1      14 -0.270883  0.147142
7       2.776161        1      15 -0.154864 -0.176970
16      2.616110        1      16 -0.022870  0.082423
18      2.157445        1      17 -0.112666 -0.076161
20      2.136019        1      18 -0.044541  0.150906
1       1.984878        1      19  0.062535  0.030486
9       1.573130        1      20  0.038836 -0.025669
5       1.569415        1      21 -0.176425 -0.126576
19      1.482985        1      22 -0.017005 -0.103987
15      1.209347        1      23 -0.296151 -0.008123
21      0.921933        1      24 -0.113675  0.042615
11      0.557585        1      25 -0.224919  0.085185, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
1360   Default  2199.000000         god  2199.000000  30.0000  30.0000
257    Default  1525.000000        file  1525.000000  29.0000  29.0000
599    Default  1082.000000         key  1082.000000  28.0000  28.0000
931    Default  1128.000000       drive  1128.000000  27.0000  27.0000
146    Default  3985.000000      people  3985.000000  26.0000  26.0000
78     Default  1264.000000      thanks  1264.000000  25.0000  25.0000
120    Default  2403.000000        good  2403.000000  24.0000  24.0000
4609   Default   836.000000         b8f   836.000000  23.0000  23.0000
25     Default  1451.000000      please  1451.000000  22.0000  22.0000
669    Default  1039.000000       space  1039.000000  21.0000  21.0000
182    Default  3168.000000         use  3168.000000  20.0000  20.0000
395    Default  1267.000000        year  1267.000000  19.0000  19.0000
350    Default  6437.000000       would  6437.000000  18.0000  18.0000
321    Default  1763.000000        said  1763.000000  17.0000  17.0000
18     Default  3432.000000        know  3432.000000  16.0000  16.0000
2      Default  1634.000000      anyone  1634.000000  15.0000  15.0000
142    Default  2524.000000         new  2524.000000  14.0000  14.0000
2940   Default   765.000000         mr.   765.000000  13.0000  13.0000
679    Default   813.000000        team   813.000000  12.0000  12.0000
1786   Default   776.000000        game   776.000000  11.0000  11.0000
1599   Default  1170.000000  government  1170.000000  10.0000  10.0000
4593   Default   685.000000         a86   685.000000   9.0000   9.0000
332    Default  2985.000000       think  2985.000000   8.0000   8.0000
1142   Default   776.000000        card   776.000000   7.0000   7.0000
130    Default  3916.000000        like  3916.000000   6.0000   6.0000
118    Default  3504.000000         get  3504.000000   5.0000   5.0000
823    Default  1356.000000     program  1356.000000   4.0000   4.0000
2992   Default   648.000000   president   648.000000   3.0000   3.0000
994    Default   948.000000         law   948.000000   2.0000   2.0000
1166   Default   505.000000       entry   505.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
10892  Topic25    31.872097        1.75    32.845581   5.1592  -5.1500
4671   Topic25    31.594999          hb    32.568481   5.1590  -5.1587
8666   Topic25